DCG function

In [ ]:
import numpy as np
import pandas as pd
import pickle

In [ ]:
pred_class = ['2', '6', '10', '12', '13', '15', '18', '19', '21', '22', '25', '26', '36', '37', '39', '48']

In [ ]:
dat = pd.read_csv('tbrain_cc_training_48tags_hash_final.csv', usecols=['dt', 'chid', 'shop_tag', 'txn_amt'])

In [ ]:
re_txn_amt = pd.DataFrame({'txn_amt':np.sort(dat['txn_amt'].unique())}).reset_index().rename(columns = {'index':'sort_txn_amt'})

dat = dat.merge(re_txn_amt, on = 'txn_amt', how = 'left')

In [ ]:
sample_dat = dat[dat['shop_tag'].isin(pred_class)]

train_dat = sample_dat[sample_dat['dt']!=24]

valid_dat = sample_dat[sample_dat['dt']==24]
valid_dat = valid_dat[valid_dat.chid.isin(train_dat.chid.unique())]

In [ ]:
def DCG(df, tag_list):
    
    dcg_sum = 0
    for i,tag in enumerate(tag_list):
        if tag in df['shop_tag'].values:
            v = df[df['shop_tag']==tag]['txn_amt'].values[0]
        else:
            v = 0
        dcg_sum += v/np.log2(2+i)
    
    return dcg_sum

In [ ]:
%%time

gd_tag = {}
for chid, group_df in valid_dat.groupby('chid')[['txn_amt', 'shop_tag']]:
    
    tag= group_df.sort_values('txn_amt', ascending = False)['shop_tag'].head(3).values
    gd_tag[chid] = [chid, tag, DCG(group_df, tag)]

In [ ]:
gd_df = pd.DataFrame(gd_tag.values(), columns = ['chid', 'tag_list', 'iDCG'])

In [ ]:
gd_dict = {}
for i, row in  gd_df.iterrows():
    gd_dict[row['chid']] = row['iDCG']

In [ ]:
with open('m24_iDCG.pkl', 'wb') as f:
    pickle.dump([gd_dict, valid_dat], f)